# Multi-ConvNet Architectures

In this notebook, we concatenate *multiple convolutional nets together* to classify IMDB movie reviews by their sentiment.

#### Load dependencies

In [5]:
import keras
from keras.datasets import imdb
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model # new! 
from keras.layers import Dense, Flatten, Dropout, Embedding, Conv1D, MaxPooling1D
from keras.layers import Input # new! 

#### Set hyperparameters

In [2]:
# training:
epochs = 10
batch_size = 128

# vector-space embedding: 
n_dim = 64
n_unique_words = 5000 # as per Maas et al. (2011); may not be optimal
n_words_to_skip = 50 # ditto
max_review_length = 100
pad_type = trunc_type = 'pre'

# convolutional layer architectures:
n_conv_1 = n_conv_2 = n_conv_3 = 128 
k_conv_1 = 2
k_conv_2 = 3
k_conv_3 = 4

# dense layer architecture: 
n_dense = 64
dropout = 0.5

#### Load data

In [3]:
(x_train, y_train), (x_valid, y_valid) = imdb.load_data(num_words=n_unique_words, skip_top=n_words_to_skip) 

#### Preprocess data

In [4]:
x_train = pad_sequences(x_train, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)
x_valid = pad_sequences(x_valid, maxlen=max_review_length, padding=pad_type, truncating=trunc_type, value=0)

#### Design neural network architecture

In [ ]:
model.add(Embedding(n_unique_words, n_dim, input_length=max_review_length)) # remove input_length
model.add(Conv1D(n_conv, k_conv, activation='relu'))
# model.add(MaxPooling1D(pool_size=2)) # doesn't help or hurt performance, but reduces parameters
model.add(Flatten())
model.add(Dense(n_dense, activation='relu'))
model.add(Dropout(dropout))
model.add(Dense(1, activation='sigmoid'))

In [6]:
input_layer = Input(shape=(max_review_length,), dtype='int16') # supports integers up to 32.7k 
embedding_layer = Embedding(n_unique_words, n_dim, input_length=max_review_length)(input_layer)

convnet_1 = Conv1D(n_conv_1, k_conv_1, activation='relu')(embedding_layer)
# max pool here
flatten_1 = Flatten()(convnet_1)

# concat here 

dense_layer = Dense(n_dense, activation='relu')(flatten_1)
dropout_layer = Dropout(dropout)(dense_layer)
# second dense + dropout here

predictions = Dense(1, activation='sigmoid')(dropout_layer)

model = Model(input_layer, predictions)

In [7]:
model.summary() 

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 64)           320000    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 99, 128)           16512     
_________________________________________________________________
flatten_1 (Flatten)          (None, 12672)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                811072    
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total para

#### Configure model

In [8]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### Train!

In [9]:
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_valid, y_valid))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 8s - loss: 0.5140 - acc: 0.7208 - val_loss: 0.3512 - val_acc: 0.8441
Epoch 2/10
25000/25000 [==============================] - 8s - loss: 0.3088 - acc: 0.8761 - val_loss: 0.3467 - val_acc: 0.8470
Epoch 3/10
25000/25000 [==============================] - 7s - loss: 0.2522 - acc: 0.9044 - val_loss: 0.3652 - val_acc: 0.8437
Epoch 4/10
25000/25000 [==============================] - 8s - loss: 0.1858 - acc: 0.9318 - val_loss: 0.4267 - val_acc: 0.8324
Epoch 5/10
25000/25000 [==============================] - 8s - loss: 0.1135 - acc: 0.9629 - val_loss: 0.5116 - val_acc: 0.8311
Epoch 6/10
25000/25000 [==============================] - 7s - loss: 0.0507 - acc: 0.9862 - val_loss: 0.6505 - val_acc: 0.8270
Epoch 7/10
18048/25000 [====================>.........] - ETA: 1s - loss: 0.0203 - acc: 0.9958

KeyboardInterrupt: 